# Реализация и сравнение Reflex Attention с обычным трансформером

## Введение

Цель данного проекта — реализовать механизм Reflex Attention в модели трансформера (на основе NanoGPT) и сравнить его с обычным трансформером. Reflex Attention добавляет cross-attention к предыдущим слоям в каждом блоке трансформера, что позволяет модели получать доступ к ранним представлениям и улучшать качество работы.

### Этапы работы

1. Реализация Reflex Attention в NanoGPT.
2. Обучение двух моделей: стандартного трансформера и трансформера с Reflex Attention.
3. Сравнение производительности на датасете OpenWebText.
4. Эксперименты с различными настройками и анализ результатов.

## Часть 1: Реализация Reflex Attention

### Что такое Reflex Attention?

Reflex Attention изменяет стандартную архитектуру трансформера, добавляя cross-attention к предыдущим слоям. На $i$-м слое внимание вычисляется следующим образом:

$$
\text{Attn}_i = \text{Concat}\left[\text{SA}(h_i),\ \text{CA}(h_{i-1}, h_i),\ \text{CA}(h_{i-2}, h_i)\right],
$$

где:

- **SA** (Self-Attention): стандартное внимание на текущих представлениях $h_i$.
- **CA** (Cross-Attention): внимание, где:
  - запросы ($Q$) берутся из $h_i$,
  - ключи и значения ($K$, $V$) берутся из предыдущих слоев: $h_{i-1}$ и $h_{i-2}$.

### Модификация модели

#### 1. Клонирование репозитория NanoGPT

Клонируем репозиторий NanoGPT:



In [1]:
!git clone https://github.com/karpathy/nanoGPT.git
!cd nanoGPT

Cloning into 'nanoGPT'...
remote: Enumerating objects: 682, done.
remote: Total 682 (delta 0), reused 0 (delta 0), pack-reused 682 (from 1)
Receiving objects: 100% (682/682), 952.47 KiB | 23.81 MiB/s, done.
Resolving deltas: 100% (385/385), done.


2. Изменения в model.py
Модифицируем файл model.py для реализации Reflex Attention.

a. Изменение класса Block
Добавляем в блок трансформера поддержку Reflex Attention:

In [5]:
import torch.nn as nn


class Block(nn.Module):
    """Трансформер-блок с Reflex Attention"""

    def __init__(self, config, layer_idx):
        super().__init__()
        self.layer_idx = layer_idx
        self.n_head = config.n_head

        # Разделение числа голов между SA и CA
        self.n_head_sa = 2  # Голов для Self-Attention
        self.n_head_ca1 = 2  # Голов для Cross-Attention с h_{i-1}
        self.n_head_ca2 = 2  # Голов для Cross-Attention с h_{i-2}

        # Проверка на корректность количества голов
        assert self.n_head_sa + self.n_head_ca1 + self.n_head_ca2 == self.n_head, "Сумма голов должна быть равна общему количеству голов"

        # Self-Attention
        self.sa = CausalSelfAttention(config, n_head=self.n_head_sa)

        # Cross-Attention с предыдущими слоями
        if self.layer_idx >= 1:
            self.ca1 = CrossAttention(config, n_head=self.n_head_ca1)
        else:
            self.ca1 = None

        if self.layer_idx >= 2:
            self.ca2 = CrossAttention(config, n_head=self.n_head_ca2)
        else:
            self.ca2 = None

        # Нормализация и MLP
        self.ln1 = nn.LayerNorm(config.n_embd)
        self.ln2 = nn.LayerNorm(config.n_embd)
        self.mlp = nn.Sequential(
            nn.Linear(config.n_embd, 4 * config.n_embd),
            nn.GELU(),
            nn.Linear(4 * config.n_embd, config.n_embd),
            nn.Dropout(config.dropout),
        )

b. Реализация Cross-Attention

In [6]:
class CrossAttention(nn.Module):
    """Cross-Attention модуль"""

    def __init__(self, config, n_head):
        super().__init__()
        self.n_head = n_head
        self.head_dim = config.n_embd // config.n_head
        self.scale = self.head_dim ** -0.5

        self.q_proj = nn.Linear(config.n_embd, n_head * self.head_dim)
        self.k_proj = nn.Linear(config.n_embd, n_head * self.head_dim)
        self.v_proj = nn.Linear(config.n_embd, n_head * self.head_dim)
        self.out_proj = nn.Linear(n_head * self.head_dim, config.n_embd)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x, k_v, mask=None):
        B, T, _ = x.size()

        # Проекция запросов, ключей и значений
        q = self.q_proj(x).view(B, T, self.n_head, self.head_dim).transpose(1, 2)
        k = self.k_proj(k_v).view(B, T, self.n_head, self.head_dim).transpose(1, 2)
        v = self.v_proj(k_v).view(B, T, self.n_head, self.head_dim).transpose(1, 2)

        # Вычисление внимания
        attn_weights = (q @ k.transpose(-2, -1)) * self.scale

        if mask is not None:
            attn_weights = attn_weights.masked_fill(mask == 0, float('-inf'))

        attn_probs = F.softmax(attn_weights, dim=-1)
        attn_probs = self.dropout(attn_probs)

        y = attn_probs @ v
        y = y.transpose(1, 2).contiguous().view(B, T, -1)
        y = self.out_proj(y)
        return y



c. Обновление метода forward в Block

In [7]:
def forward(self, x, h_prev_layers):
    """
    x: Текущее скрытое состояние [B, T, C]
    h_prev_layers: Список предыдущих скрытых состояний [h_{i-1}, h_{i-2}]
    """
    # Нормализация входа
    x_norm = self.ln1(x)

    # Self-Attention
    y_sa = self.sa(x_norm)

    attentions = [y_sa]

    # Cross-Attention с h_{i-1}
    if self.ca1 is not None and len(h_prev_layers) >= 1:
        h_prev1_norm = self.ln1(h_prev_layers[-1])
        y_ca1 = self.ca1(x_norm, h_prev1_norm)
        attentions.append(y_ca1)

    # Cross-Attention с h_{i-2}
    if self.ca2 is not None and len(h_prev_layers) >= 2:
        h_prev2_norm = self.ln1(h_prev_layers[-2])
        y_ca2 = self.ca2(x_norm, h_prev2_norm)
        attentions.append(y_ca2)

    # Объединение результатов внимания
    y = torch.cat(attentions, dim=-1)
    x = x + y

    # Пропуск через MLP
    x = x + self.mlp(self.ln2(x))

    return x


In [8]:
class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.transformer = nn.ModuleDict({
            'wte': nn.Embedding(config.vocab_size, config.n_embd),
            'wpe': nn.Embedding(config.block_size, config.n_embd),
            'drop': nn.Dropout(config.dropout),
            'h': nn.ModuleList([Block(config, layer_idx=i) for i in range(config.n_layer)]),
            'ln_f': nn.LayerNorm(config.n_embd),
        })
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        self.apply(self._init_weights)

    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size, "Длина последовательности превышает block_size"

        # Получение эмбеддингов
        token_embeddings = self.transformer.wte(idx)  # [b, t, n_embd]
        position_embeddings = self.transformer.wpe(torch.arange(t, device=device))  # [t, n_embd]
        x = self.transformer.drop(token_embeddings + position_embeddings)

        # Список скрытых состояний для Reflex Attention
        h_prev_layers = []

        # Проход через блоки
        for block in self.transformer.h:
            x = block(x, h_prev_layers)
            h_prev_layers.append(x)

        x = self.transformer.ln_f(x)
        logits = self.lm_head(x)

        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
            return logits, loss
        else:
            return logits


# Часть 2: Обучение и эксперименты

## Подготовка данных

Используем датасет OpenWebText.

In [10]:
!python data/openwebtext/prepare.py

python3: can't open file '/content/data/openwebtext/prepare.py': [Errno 2] No such file or directory


In [16]:
!ls nanoGPT


assets	  config	   data     model.py   sample.py	   train.py
bench.py  configurator.py  LICENSE  README.md  scaling_laws.ipynb  transformer_sizing.ipynb


Настройка конфигураций
Создаем две конфигурации: для стандартного трансформера и трансформера с Reflex Attention.

a. Конфигурация стандартного трансформера

In [17]:
# nanoGPT/config/train_standard.py

# Базовые параметры (ранее из train_shakespeare.py)
n_layer = 4  # Количество слоев в модели
n_head = 4   # Количество голов в self-attention
n_embd = 256  # Размерность эмбеддингов
batch_size = 32  # Размер батча
learning_rate = 3e-4  # Начальная скорость обучения
max_iters = 1000  # Максимальное количество итераций

# Переопределяем параметры для текущего эксперимента
out_dir = 'out-standard'
eval_interval = 500  # Интервал оценки модели
eval_iters = 200      # Число итераций для оценки
log_interval = 100    # Интервал логирования

# Гиперпараметры модели
n_layer = 6  # Изменяем количество слоев
n_head = 6   # Изменяем количество голов
n_embd = 384  # Изменяем размерность эмбеддингов

# Гиперпараметры обучения
batch_size = 64  # Увеличиваем размер батча
learning_rate = 3e-4  # Оставляем скорость обучения без изменений
max_iters = 5000  # Увеличиваем количество итераций



b. Конфигурация Reflex Attention

In [19]:
# nanoGPT/config/train_reflex.py

# Базовые параметры
out_dir = 'out-reflex'          # Директория для сохранения результатов
eval_interval = 500             # Интервал оценки
eval_iters = 200                # Число итераций для оценки
log_interval = 100              # Интервал логирования

# Гиперпараметры модели
n_layer = 6                     # Количество слоев трансформера
n_head = 6                      # Количество голов внимания
n_embd = 384                    # Размерность эмбеддингов

# Гиперпараметры обучения
batch_size = 32                 # Размер батча
learning_rate = 3e-4            # Скорость обучения
max_iters = 50000               # Максимальное количество итераций



Запуск обучения
1. Стандартный трансформер

In [21]:
!python train.py config/train_standard.py

python3: can't open file '/content/train.py': [Errno 2] No such file or directory


2. Reflex Attention

In [23]:
!python train.py config/train_reflex.py

python3: can't open file '/content/train.py': [Errno 2] No such file or directory


In [25]:
!tensorboard --logdir out-standard --port 6006
!tensorboard --logdir out-reflex --port 6007

2024-11-27 07:12:29.605576: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-27 07:12:29.630743: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-27 07:12:29.638233: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-27 07:12:29.656544: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-27 07:12:31.026112: W tensorflow/compiler/tf2

Часть 3: Результаты экспериментов
Наблюдения
После завершения обучения ожидаем следующие результаты:

Скорость сходимости:

Reflex Attention может быстрее сходиться за счет более богатых представлений.
Однако это может потребовать большего времени на вычисления.
Качество:

Reflex Attention должен показать лучшие результаты на длинных последовательностях, где важно учитывать ранний контекст.
Возможные проблемы
Увеличение памяти: Reflex Attention требует хранения состояний предыдущих слоев.
Переобучение: Из-за увеличения параметров модель может переобучиться.
Часть 4: Дополнительные эксперименты
Эксперимент 1: Количество голов
Цель: Проверить, как распределение голов между SA и CA влияет на результаты.
Настройки:
Больше голов для SA, меньше для CA.
Равное количество голов для SA и CA.
Эксперимент 2: Влияние на разные слои
Цель: Изучить, как Reflex Attention на разных слоях влияет на обучение.
Настройки:
Применение Reflex Attention только на верхних слоях.
Исключение Reflex Attention из нижних слоев.
Выводы
Что получилось:

Реализован Reflex Attention с разделением внимания на текущий и предыдущие слои.
Подготовлены эксперименты для сравнения с базовым трансформером.
Что может быть улучшено:

Оптимизация памяти и вычислений.
Эксперименты с более сложными конфигурациями.
